# Cubic Spline Path Optimization
## Offline Path Planning Optimization

In [27]:
%matplotlib widget

import time
import nlopt
import math
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import scipy.special
from cubic_bezier_planner import calc_bezier_path
from cubic_spline_planner import calc_spline_course

In [28]:
class Path:
    def __init__(self, x, y, yaw, k):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.k = k

## Spline Class
A spline class is initated with an input set of waypoints and a maximum deviation. Boundary lines are constructed, and this class features functions for calculating derivatives, yaw, curvature and path distance.

In [29]:
class Spline():

    def __init__(self, ax, ay, bound):
        
        #input waypoint coordinates
        ayaw, k = self.calc_yaw_curvature(ax, ay)
        self.waypoints = Path(ax, ay, ayaw, k)
        
        # defines and sets left and right boundary lines
        self.bound = bound
        lax, lay, rax, ray = self.init_boundary()
        self.left_bound = Path(lax, lay, None, None)
        self.right_bound = Path(rax, ray, None, None)

        # default and seeding path
        cx, cy, cyaw, ck, _ = calc_spline_course(ax, ay, 0.5)
        self.default_path = Path(cx, cy, cyaw, ck)
        self.seeding_path = self.default_path
        self.optimized_path = Path([], [], [], [])


    # Calculates the first derivative of input arrays
    def calc_d(self, x, y):

        dx, dy = [], []

        for i in range(0, len(x)-1):
            dx.append(x[i+1] - x[i])
            dy.append(y[i+1] - y[i])
        
        dx.append(dx[-1])
        dy.append(dy[-1])
        return dx, dy

    # Calculates yaw and curvature given input path
    def calc_yaw_curvature(self, x, y):

        dx, dy = self.calc_d(x,y)
        ddx, ddy = self.calc_d(dx, dy)
        yaw = []
        k = []

        for i in range(0, len(x)):
            yaw.append(math.atan2(dy[i], dx[i]))
            k.append( (ddy[i] * dx[i] - ddx[i] * dy[i]) / ((dx[i]**2 + dy[i]**2)**(3/2)) )
    
        return yaw, k

    # Calculates total distance of the path
    def calc_path_dist(self, x, y):

        dx, dy = self.calc_d(x, y)
        dx = np.absolute(dx)
        dy = np.absolute(dy)
        ddist = np.hypot(dx, dy)

        return np.sum(ddist)


    # Determines position of boundary lines for visualization
    def init_boundary(self):

        dyaw = self.waypoints.yaw.copy()
        for n in range(1, len(ax)-1):
            self.waypoints.yaw[n] = 0.5*(dyaw[n] + dyaw[n-1])

        rax, ray, lax, lay = [], [], [], []

        for n in range(0, len(self.waypoints.yaw)):
            lax.append(self.waypoints.x[n] - self.bound*np.sin(self.waypoints.yaw[n]))
            lay.append(self.waypoints.y[n] + self.bound*np.cos(self.waypoints.yaw[n]))
            rax.append(self.waypoints.x[n] + self.bound*np.sin(self.waypoints.yaw[n]))
            ray.append(self.waypoints.y[n] - self.bound*np.cos(self.waypoints.yaw[n]))
        
        return lax, lay, rax, ray


## Optimization Strategy

Both methods may be optimized with the same desired path properties, essentially minimizing curvature and variation of curvature and distance if required. The cubic Bezier curve has an additional cost penalizing the curvature discontinuity at the point of joining two subsequent Bezier curves.

In [30]:
class Spline(Spline):

    def optimize_min_spline(self):

        self.default_path = self.seeding_path

        # number of points
        N = len(self.seeding_path.yaw)
        print("Optimizing {} points".format(N))

        # construct initial guess
        initial_guess = np.zeros(N-2)

        # construct bound
        bound = []
        for _ in range(N-2):
            bound.append((-self.bound, self.bound))
        bnds = tuple(bound)

        result = optimize.minimize(self.spline_objective_func, initial_guess, bounds=bnds)
    
        cx = self.seeding_path.x.copy()
        cy = self.seeding_path.y.copy()
        
        
        if result.success:
            print("optimized true")
            offsets = result.x
            
            # updated set of waypoints
            for n in range(N-2):
                cx[n+1] -= offsets[n]*np.sin(self.seeding_path.yaw[n+1])
                cy[n+1] += offsets[n]*np.cos(self.seeding_path.yaw[n+1])

            cyaw, ck = self.calc_yaw_curvature(cx, cy)

            self.optimized_path = Path(cx, cy, cyaw, ck)    

        else:
            print("optimization failure, defaulting")
            exit()

    # Objective function for quintic bezier
    def spline_objective_func(self, offsets):

        cx = self.seeding_path.x.copy()
        cy = self.seeding_path.y.copy()

        for n in range(len(cx)-2):
            cx[n+1] -= offsets[n]*np.sin(self.seeding_path.yaw[n+1])
            cy[n+1] += offsets[n]*np.cos(self.seeding_path.yaw[n+1])
        
        yaw, k = self.calc_yaw_curvature(cx, cy)
        absolute_k = np.square(k)
        curvature_cost = 1.0 * np.mean(absolute_k)

        dk, _ = self.calc_d(yaw, k)
        absolute_dk = np.square(dk)
        continuity_cost = 1.0 * np.mean(absolute_dk)

        distance_cost = 0.5 * self.calc_path_dist(cx, cy)

        absolute_offset = np.absolute(offsets)
        offset_cost = 0.1 * np.mean(absolute_offset)

        return curvature_cost + offset_cost + distance_cost + continuity_cost



In [31]:
    # define input path
    ay = [0.0, 2.3, 6.25, 8.6, 8.2, 5.3, 2.6]
    ax = [0.0, 7.16, 13.68, 22.3, 30.64, 39.6, 50.4]
    boundary = 1.0

    spline = Spline(ax, ay, boundary)

    t0 = time.process_time()
    spline.optimize_min_spline()
    t1 = time.process_time() - t0
    print("Solved in: ", t1)

    # Path plot
    plt.subplots(1)
    plt.plot(spline.left_bound.x, spline.left_bound.y, '--r', alpha=0.5, label="left boundary")
    plt.plot(spline.right_bound.x, spline.right_bound.y, '--g', alpha=0.5, label="right boundary")
    plt.plot(spline.default_path.x, spline.default_path.y, '-y', label="default")
    plt.plot(spline.optimized_path.x, spline.optimized_path.y, '-m', label="optimized")
    plt.plot(spline.waypoints.x, spline.waypoints.y, '.', label="waypoints")
    plt.grid(True)
    plt.legend()
    plt.axis("equal")

    # Heading plot
    plt.subplots(1)
    plt.plot([np.rad2deg(iyaw) for iyaw in spline.default_path.yaw], "-y", label="original")
    plt.plot([np.rad2deg(iyaw) for iyaw in spline.optimized_path.yaw], "-m", label="optimized")
    plt.grid(True)
    plt.legend()
    plt.xlabel("line length[m]")
    plt.ylabel("yaw angle[deg]")

    # Curvature plot
    plt.subplots(1)
    plt.plot(spline.default_path.k, "-y", label="original")
    plt.plot(spline.optimized_path.k, "-m", label="optimized")
    plt.grid(True)
    plt.legend()
    plt.xlabel("line length[m]")
    plt.ylabel("curvature [1/m]")

    plt.show()

Optimizing 106 points
optimized true
Solved in:  6.794625410999998


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …